In [ ]:
!git clone https://github.com/agentmorris/MegaDetector.git



**GROUP #12**

**Muhammad Abubakar Minhas**

**Muhammad Rayed**






**Our reasons for choosing this dataset(LILA) and our ideas and insights for how this can can be useful:**


Our discussion with Sir told us that we can pursue 2 directions in this project, working on better detection or better classification. He also advised us to keep the scope of the project to identifying big cats, since snow leopard images are endangered anyways so collecting a substantial amount of snow leopard images might not be possible. 

First of all, we chose the LILA dataset as this is the only dataset that contains cameratrap images. We had explored Camouflaged Object Detection before and we wrote about it in our first deliverable as well, but we have come to realise that COD datasets, while quite large(COD10K for e.g.), are not representative of the IRL images obtained from the cameras in Margalla because of many reasons: COD images are ideal high res images taken professionally. A major drawback of them is that the animal is always at the center of the image, which is what a model trained on these images learns. We were thinking of generation techniques to maybe shift the images so that the model learns to find the animal in the periphery as well, but then Sir told us that the problem with the project deployed in Margalla is that the model(which is Megadetector based), has mostly been trained on non empty images(although we did find several images in this dataset as you can see below that are empty, so we need more guidance regarding that), in which the animal is the only significant object, so it incorrectly classifies any significant object that it sees as a snow leopard even when there is no snow leopard(which is mostly the case). Some examples that Sir gave was an amplified rain drop due to motion blur, and spiderwebs. 

**Our ideas:**

Disclaimer: We are not aware of the feasability of our ideas yet, these are just some techniques that we thought we could apply to the images of big cats from in LILA.

Tweaking the training images in the LILA dataset to make them similar to the unideal images that we get from the cameras in Margalla. Firstly, as Sir told us that the megadetector has not been trained on empty images, we had an idea of generating an empty image by removing the big cat from it. We are not sure how we will accomplish this as if we simply cut out the bounding box of the cat and replace it with a blackbox, the model will learn that the difference between an empty image and one with a leopard is a blackbox, which will obviously lead to missclassification. Moreover, we could apply some sort of averaging filter(but we'd have to make sure that the blackness of the bounnding box is not averaged onto the image) or maybe a GAN to generate the empty pixels based on the rest of the image.

Moreover, we are not sure how feasable generation will be in order to generate images with noise such as raindrops and spiderwebs.


**About this notebook:**

This notebook has been imported from the Megadetector Github Repository. We tried to download the zip file but it was obviously way too large(the annotations file only was 2 GB), so we found an article written by Dan Morris(the guy who made megadetector) in the website he made fro LILA, which instructed us to use different methods to download a subset of the data instead, for which he had pushed a notebook on Github. That too, was not feasable as the table containing the URLS and annotations of the images would take up all available RAM of T4 on both Kaggle and Google Colab while downloading, and the kernel crashed before the download would be complete. So we found another file instead in the same repo which gives the count of each class in each subdataset of LILA. This notebook is that file.


**What we have found:**

**Number of big cat images per dataset(showing 3 of 24)**

**Caltech Camera Traps**

Bobcat - 8098

Mountain Lion - 145

**Snapshot Serengeti**

lionfemale - 16605

cheetah - 6806

lionmale - 5488

leopard - 650

wildcat - 193

lioncub - 73

**WCS Camera Traps**

Panthera onca (Jaguar) - 9,435

Panthera pardus (Leopard) - 470

Panthera leo (Lion) - 429

Panthera tigris (Tiger) - 321

Puma concolor (Cougar/Mountain Lion) - 6,823

Acinonyx jubatus (Cheetah) - 54

Neofelis nebulosa (Clouded Leopard) - 204

Neofelis diardi (Sunda Clouded Leopard) - 76

Caracal caracal (Caracal) - 36




Moving forward, we aim to first actually obtain these images for which we will be looking into Collab and Kaggle Premium. After obtaining the dataset, we could potentially apply generation techniques as discussed before to add noise in the images and make them less ideal.











In [23]:
!sed -i "s/import winreg/try:\n    import winreg\nexcept ImportError:\n    winreg = None/" /usr/local/lib/python3.10/dist-packages/megadetector/utils/path_utils.py

In [24]:
%cd MegaDetector
!pip install .

[Errno 2] No such file or directory: 'MegaDetector'
/kaggle/working/MegaDetector
Processing /kaggle/working/MegaDetector
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for megadetector: filename=megadetector-5.0.25-py3-none-any.whl size=783340 sha256=146598d1c64300be54d248198f59f940dee22d35a0eb17c3c9423f6408988402
  Stored in directory: /tmp/pip-ephem-wheel-cache-q6v5zk6p/wheels/7f/58/88/750512f3057911068d4535dc4cccff43d59fedecc92e75ff0d
Successfully built megadetector
  Attempting uninstall: megadetector
    Found existing installation: megadetector 5.0.25
    Uninstalling megadetector-5.0.25:
      Successfully uninstalled megadetector-5.0.25


In [26]:
!sed -i "s/import winreg/try:\n    import winreg\nexcept ImportError:\n    winreg = None/" /usr/local/lib/python3.10/dist-packages/megadetector/utils/path_utils.py

In [27]:
"""

get_lila_annotation_counts.py

Generates a .json-formatted dictionary mapping each LILA dataset to all categories
that exist for that dataset, with counts for the number of occurrences of each category
(the number of *annotations* for each category, not the number of *images*).

Also loads the taxonomy mapping file, to include scientific names for each category.

get_lila_image_counts.py counts the number of *images* for each category in each dataset.

"""

#%% Constants and imports

import json
import os

from megadetector.data_management.lila.lila_common import \
    read_lila_metadata, read_metadata_file_for_dataset, read_lila_taxonomy_mapping

# cloud provider to use for downloading images; options are 'gcp', 'azure', or 'aws'
preferred_cloud = 'gcp'

# array to fill for output
category_list = []

# We'll write images, metadata downloads, and temporary files here
lila_local_base = os.path.expanduser('~/lila')

output_dir = os.path.join(lila_local_base,'lila_categories_list')
os.makedirs(output_dir,exist_ok=True)

metadata_dir = os.path.join(lila_local_base,'metadata')
os.makedirs(metadata_dir,exist_ok=True)

output_file = os.path.join(output_dir,'lila_dataset_to_categories.json')


#%% Load category and taxonomy files

taxonomy_df = read_lila_taxonomy_mapping(metadata_dir)


#%% Map dataset names and category names to scientific names

ds_query_to_scientific_name = {}

unmapped_queries = set()

datasets_with_taxonomy_mapping = set()

# i_row = 1; row = taxonomy_df.iloc[i_row]; row
for i_row,row in taxonomy_df.iterrows():

    datasets_with_taxonomy_mapping.add(row['dataset_name'])

    ds_query = row['dataset_name'] + ':' + row['query']
    ds_query = ds_query.lower()

    if not isinstance(row['scientific_name'],str):
        unmapped_queries.add(ds_query)
        ds_query_to_scientific_name[ds_query] = 'unmapped'
        continue

    ds_query_to_scientific_name[ds_query] = row['scientific_name']

print('Loaded taxonomy mappings for {} datasets'.format(len(datasets_with_taxonomy_mapping)))


#%% Download and parse the metadata file

metadata_table = read_lila_metadata(metadata_dir)

print('Loaded metadata URLs for {} datasets'.format(len(metadata_table)))


#%% Download and extract metadata for each dataset

for ds_name in metadata_table.keys():
    metadata_table[ds_name]['json_filename'] = read_metadata_file_for_dataset(ds_name=ds_name,
                                                                         metadata_dir=metadata_dir,
                                                                         metadata_table=metadata_table)


#%% Get category names and counts for each dataset

# Takes ~5 minutes

from collections import defaultdict

dataset_to_categories = {}

# ds_name = 'NACTI'
for ds_name in metadata_table.keys():

    taxonomy_mapping_available = (ds_name in datasets_with_taxonomy_mapping)

    if not taxonomy_mapping_available:
        print('Warning: taxonomy mapping not available for {}'.format(ds_name))

    print('Finding categories in {}'.format(ds_name))

    json_filename = metadata_table[ds_name]['json_filename']
    base_url = metadata_table[ds_name]['image_base_url_' + preferred_cloud]
    assert not base_url.endswith('/')

    # Open the metadata file
    with open(json_filename, 'r') as f:
        data = json.load(f)

    # Collect list of categories and mappings to category name
    categories = data['categories']

    category_id_to_count = defaultdict(int)
    annotations = data['annotations']

    # ann = annotations[0]
    for ann in annotations:
        category_id_to_count[ann['category_id']] = category_id_to_count[ann['category_id']] + 1

    # c = categories[0]
    for c in categories:
       count = category_id_to_count[c['id']]
       if 'count' in c:
           assert 'bbox' in ds_name or c['count'] == count
       c['count'] = count

       # Don't do taxonomy mapping for bbox data sets, which are sometimes just binary and are
       # always redundant with the class-level data sets.
       if 'bbox' in ds_name:
           c['scientific_name_from_taxonomy_mapping'] = None
       elif not taxonomy_mapping_available:
           c['scientific_name_from_taxonomy_mapping'] = None
       else:
           taxonomy_query_string = ds_name.lower().strip() + ':' + c['name'].lower()
           if taxonomy_query_string not in ds_query_to_scientific_name:
               print('No match for query string {}'.format(taxonomy_query_string))
               # As of right now, this is the only quirky case
               assert '#ref!' in taxonomy_query_string and 'wcs' in ds_name.lower()
               c['scientific_name_from_taxonomy_mapping'] = None
           else:
               sn = ds_query_to_scientific_name[taxonomy_query_string]
               assert sn is not None and len(sn) > 0
               c['scientific_name_from_taxonomy_mapping'] = sn

    dataset_to_categories[ds_name] = categories

# ...for each dataset


#%% Print the results

# ds_name = list(dataset_to_categories.keys())[0]
for ds_name in dataset_to_categories:

    print('\n** Category counts for {} **\n'.format(ds_name))

    categories = dataset_to_categories[ds_name]
    categories = sorted(categories, key=lambda x: x['count'], reverse=True)

    for c in categories:
        print('{} ({}): {}'.format(c['name'],c['scientific_name_from_taxonomy_mapping'],c['count']))

# ...for each dataset


#%% Save the results

with open(output_file, 'w') as f:
    json.dump(dataset_to_categories,f,indent=1)


Loaded taxonomy mappings for 24 datasets
Loaded metadata URLs for 24 datasets
Finding categories in Caltech Camera Traps
Finding categories in ENA24
Finding categories in Missouri Camera Traps
Finding categories in NACTI
Finding categories in WCS Camera Traps
No match for query string wcs camera traps:#ref!
Finding categories in Wellington Camera Traps
Finding categories in Island Conservation Camera Traps
Finding categories in Channel Islands Camera Traps
Finding categories in Idaho Camera Traps
Finding categories in Snapshot Serengeti
Finding categories in Snapshot Karoo
Finding categories in Snapshot Kgalagadi
Finding categories in Snapshot Enonkishu
Finding categories in Snapshot Camdeboo
Finding categories in Snapshot Mountain Zebra
Finding categories in Snapshot Kruger
Finding categories in SWG Camera Traps
Finding categories in Orinoquia Camera Traps
Finding categories in Trail Camera Images of New Zealand Animals
Finding categories in Desert Lion Conservation Camera Traps
Findi